# Importing the datasets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import spacy
# Download the en_core_web_sm model
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep='\t',header=None)
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
data_imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)

In [5]:
data_yelp.shape, data_amazon.shape, data_imdb.shape

((1000, 2), (1000, 2), (748, 2))

In [6]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [9]:
# Assign column names
column_name = ['Review', 'Sentiment']
data_yelp.columns = column_name
data_amazon.columns = column_name
data_imdb.columns = column_name

In [13]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [15]:
data = pd.concat([data_yelp,data_amazon, data_imdb],ignore_index=True)

In [17]:
data.shape

(2748, 2)

In [19]:
data['Sentiment'].value_counts()

# 1386 positive reviews
# 1362 Negative reviews

Sentiment
1    1386
0    1362
Name: count, dtype: int64

In [20]:
# check for null values
data.isnull().sum()

# no null values in the data

Review       0
Sentiment    0
dtype: int64

In [21]:
x = data['Review']
y = data['Sentiment']

# Data Cleaning

In [23]:
import string

In [24]:
punct = string.punctuation

In [25]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [26]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS) # list of stopwords

In [34]:
len(stopwords)


326

In [35]:
# creating a function for data cleaning
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
 
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [36]:
# if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemma for proper noun
text_data_cleaning("Hello all, It's a beautiful day outside there!")
# stopwords and punctuations removed

['hello', 'beautiful', 'day', 'outside']

# Vectorization Feature Engineering (TF-IDF)

In [37]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [38]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function

In [39]:
classifier = LinearSVC()

# Train the model

In [40]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
x_train.shape, x_test.shape
# 2198 samples in training dataset and 550 in test dataset

((2198,), (550,))

In [41]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [42]:
clf.fit(x_train, y_train)

C:\Users\Public\anaconda\envs\Current\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000002577B2015E0>)),
                ('clf', LinearSVC())])

In [43]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = clf.predict(x_test)
# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]], dtype=int64)

In [44]:
# classification_report
print(classification_report(y_test, y_pred))
# we are getting almost 77% accuracy

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [45]:
accuracy_score(y_test, y_pred)
# 76% accuracy

0.7672727272727272

In [46]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])
# output is 1, that means review is positive

array([1], dtype=int64)

In [47]:
clf.predict(["It's hard to learn new things!"])
# output is 0, that means review is Negative

array([0], dtype=int64)